<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/05_combined_model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-18 13:15:37--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11745 (11K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.47K  --.-KB/s    in 0s      

2022-06-18 13:15:37 (45.6 MB/s) - ‘utils.py’ saved [11745/11745]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 75.5 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, _ = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3508 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
target_shape = (32, 64, 1)

rgb_input = keras.Input(shape=IMG_SIZE+(3,))
hub_module = hub.KerasLayer(HUB_URL)

rgb_features = hub_module(rgb_input)
rgb_layer = keras.layers.Reshape(target_shape=target_shape)(rgb_features)

xyz_input = tfio.experimental.color.rgb_to_xyz(rgb_input)
xyz_features = hub_module(xyz_input)
xyz_layer = keras.layers.Reshape(target_shape=target_shape)(xyz_features)

hsv_input = tf.image.rgb_to_hsv(rgb_input)
hsv_features = hub_module(hsv_input)
hsv_layer = keras.layers.Reshape(target_shape=target_shape)(hsv_features)

block = keras.layers.Concatenate()([rgb_layer, xyz_layer, hsv_layer])

conv1 = keras.layers.Conv2D(filters=10,
                            kernel_size=(2,2),
                            strides=(1,2),
                            padding='same',
                            activation='relu')(block)

conv2 = keras.layers.Conv2D(filters=20,
                            kernel_size=(2,2),
                            strides=(2,2),
                            activation='relu')(conv1)

avg_pool = keras.layers.AveragePooling2D(pool_size=(2,2))(conv2)

conv3 = keras.layers.Conv2D(filters=20,
                            kernel_size=(1,1),
                            activation='relu')(avg_pool)

flatten = keras.layers.Flatten()(conv3)

output = keras.layers.Dense(N_CLASSES, activation='softmax')(flatten)

model = keras.models.Model(inputs=[rgb_input], outputs=[output])

In [10]:
model._name = 'combined-model'

In [11]:
metrics = [
            keras.metrics.AUC(name='auc', curve='PR', num_thresholds=100),
            'accuracy'
          ]

loss_fn = keras.losses.CategoricalCrossentropy()
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(1e-2, 533, .9)

model.compile(loss=loss_fn,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=metrics)

In [12]:
early_stopping = keras.callbacks.EarlyStopping(
                                              monitor='auc', 
                                              verbose=1,
                                              patience=3,
                                              restore_best_weights=True,
                                              mode='max')

check_point_path = os.path.join('./logs/checkpoints', model._name+"-{epoch:02d}.h5")
check_point = keras.callbacks.ModelCheckpoint(
                                            filepath=check_point_path,
                                            monitor='auc',
                                            save_best_only=True,
                                            mode='max')
        
callbacks = [early_stopping, check_point]

In [13]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=20,
                    class_weight=class_weight)

Epoch 1/20
519/519 [==============================] - 1840s 3s/step - loss: 15.8601 - auc: 0.7678 - accuracy: 0.6863
Epoch 2/20
519/519 [==============================] - 416s 798ms/step - loss: 10.1916 - auc: 0.8661 - accuracy: 0.7778
Epoch 3/20
519/519 [==============================] - 418s 804ms/step - loss: 8.7507 - auc: 0.8847 - accuracy: 0.7975
Epoch 4/20
519/519 [==============================] - 418s 804ms/step - loss: 7.6248 - auc: 0.9046 - accuracy: 0.8198
Epoch 5/20
519/519 [==============================] - 419s 805ms/step - loss: 6.8611 - auc: 0.9178 - accuracy: 0.8328
Epoch 6/20
519/519 [==============================] - 419s 805ms/step - loss: 6.0437 - auc: 0.9287 - accuracy: 0.8471
Epoch 7/20
519/519 [==============================] - 419s 805ms/step - loss: 5.4428 - auc: 0.9364 - accuracy: 0.8581
Epoch 8/20
519/519 [==============================] - 419s 805ms/step - loss: 5.0344 - auc: 0.9421 - accuracy: 0.8679
Epoch 9/20
519/519 [==============================] - 41

In [14]:
!cp ./logs/checkpoints/combined-model-10.h5 ./drive/MyDrive/checkpoints/combined-model-10.1.h5
!cp ./logs/checkpoints/combined-model-15.h5 ./drive/MyDrive/checkpoints/combined-model-15.1.h5
!cp ./logs/checkpoints/combined-model-20.h5 ./drive/MyDrive/checkpoints/combined-model-20.1.h5

In [15]:
history = model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=30,
                        initial_epoch=20,
                        class_weight=class_weight)

Epoch 21/30
519/519 [==============================] - 419s 805ms/step - loss: 1.6773 - auc: 0.9835 - accuracy: 0.9476
Epoch 22/30
519/519 [==============================] - 419s 804ms/step - loss: 1.6029 - auc: 0.9848 - accuracy: 0.9497
Epoch 23/30
519/519 [==============================] - 419s 804ms/step - loss: 1.4878 - auc: 0.9859 - accuracy: 0.9527
Epoch 24/30
519/519 [==============================] - 419s 804ms/step - loss: 1.4111 - auc: 0.9869 - accuracy: 0.9562
Epoch 25/30
519/519 [==============================] - 419s 805ms/step - loss: 1.3436 - auc: 0.9876 - accuracy: 0.9578
Epoch 26/30
519/519 [==============================] - 419s 804ms/step - loss: 1.2742 - auc: 0.9880 - accuracy: 0.9603
Epoch 27/30
519/519 [==============================] - 419s 805ms/step - loss: 1.2097 - auc: 0.9892 - accuracy: 0.9615
Epoch 28/30
519/519 [==============================] - 419s 805ms/step - loss: 1.1573 - auc: 0.9894 - accuracy: 0.9654
Epoch 29/30
519/519 [===========================

In [16]:
!cp ./logs/checkpoints/combined-model-25.h5 ./drive/MyDrive/checkpoints/combined-model-25.1.h5
!cp ./logs/checkpoints/combined-model-30.h5 ./drive/MyDrive/checkpoints/combined-model-30.1.h5

In [17]:
history = model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=35,
                        initial_epoch=30,
                        class_weight=class_weight)

Epoch 31/35
519/519 [==============================] - 420s 806ms/step - loss: 1.0314 - auc: 0.9908 - accuracy: 0.9685
Epoch 32/35
519/519 [==============================] - 419s 804ms/step - loss: 0.9957 - auc: 0.9910 - accuracy: 0.9689
Epoch 33/35
519/519 [==============================] - 419s 805ms/step - loss: 0.9585 - auc: 0.9913 - accuracy: 0.9704
Epoch 34/35
519/519 [==============================] - 419s 804ms/step - loss: 0.9375 - auc: 0.9919 - accuracy: 0.9712
Epoch 35/35
519/519 [==============================] - 418s 804ms/step - loss: 0.9046 - auc: 0.9921 - accuracy: 0.9721


In [18]:
!cp ./logs/checkpoints/combined-model-35.h5 ./drive/MyDrive/checkpoints/combined-model-35.1.h5